## This notebook is for dataset and benchmark creation for MoleculeACE benchmarks 

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import pandas as pd
import datamol as dm

from polaris.dataset import Dataset, ColumnAnnotation
from polaris.benchmark import SingleTaskBenchmarkSpecification
from polaris.utils.types import HubOwner


root = pathlib.Path("__file__").absolute().parents[2]
# set to recipe root directory
os.chdir(root)
sys.path.insert(0, str(root))

from utils.docs_utils import load_readme


In [2]:
# Get the owner and organization
org = "MolecularML"

owner = HubOwner(slug=org.lower(), type="organization")
owner

HubOwner(slug='molecularml', external_id=None, type='organization')

In [3]:
tags = ["Activity Cliff", "QSAR"]
source = "https://pubs.acs.org/doi/10.1021/acs.jcim.2c01073"
target_col = "y"
task = "regression"
mol_col = "smiles"

In [4]:
mapping = {
    "CHEMBL1862": "Tyrosine protein kinase Abl family",
    "CHEMBL1871": "Nuclear hormone receptor subfamily 3 group C member 4",
    "CHEMBL2034": "Nuclear hormone receptor subfamily 3 group C member 1",
    "CHEMBL2047": "Nuclear hormone receptor subfamily 1 group H member 4",
    "CHEMBL204": "Serine protease S1A subfamily",
    "CHEMBL2147": "CAMK protein kinase PIM family",
    "CHEMBL214": "Serotonin receptor",
    "CHEMBL218": "Cannabinoid receptor",
    "CHEMBL219": "Dopamine receptor",
    "CHEMBL228": "SLC06 neurotransmitter transporter family",
    "CHEMBL231": "Histamine receptor",
    "CHEMBL233": "Opioid receptor",
    "CHEMBL234": "Dopamine receptor",
    "CHEMBL235": "Nuclear hormone receptor subfamily 1 group C member 3",
    "CHEMBL236": "Opioid receptor",
    "CHEMBL237": "Opioid receptor",
    "CHEMBL238": "SLC06 neurotransmitter transporter family",
    "CHEMBL239": "Nuclear hormone receptor subfamily 1 group C member 1",
    "CHEMBL244": "Serine protease S1A subfamily",
    "CHEMBL262": "CMGC protein kinase GSK family",
    "CHEMBL264": "Histamine receptor",
    "CHEMBL2835": "Tyrosine protein kinase JakA family",
    "CHEMBL287": "Membrane receptor",
    "CHEMBL2971": "Tyrosine protein kinase JakA family",
    "CHEMBL3979": "Nuclear hormone receptor subfamily 1 group C member 2",
    "CHEMBL4005": "Transferase",
    "CHEMBL4203": "CMGC protein kinase CLK family",
    "CHEMBL4616": "GRP-related receptor",
    "CHEMBL4792": "Orexin receptor",
}

readout_map = {"Ki": "Inhibition [Inhibitory Constant, Ki]" ,
               "EC50": "Agonism [Half-Maximal Effective Concentration, EC50]"}

In [15]:
# create readme files
for path in dm.fs.glob(
    f"org-MolecularML/moleculeACE/benchmark_data/*.csv"
):
    table = dm.read_csv(path)
    name = pathlib.Path(path).stem
    chembl_id, readout = name.split("_")
    target_name = mapping[chembl_id]
    dataset_name = f"moleculeace_{name}"
    benchmark_name = dataset_name
    chembl_id, readout = name.split("_")

    readout_desc = readout_map[readout]
    readme = f'## Background\nThis dataset comprises collected and curated bioactivity data for the target {target_name} from ChEMBL assay {chembl_id}, utilized to evaluate the performance of various machine learning algorithms on activity cliffs. We employed classical machine learning methods combined with common molecular descriptors, as well as neural networks based on unstructured molecular data such as molecular graphs or SMILES strings.\n\nActivity cliffs are molecules with small differences in structure but large differences in potency. Activity cliffs play an important role in drug discovery, but the bioactivity of activity cliff compounds are notoriously difficult to predict.\n\n## Description of readouts\n- `exp_mean [nM]`: {readout_desc}\n- `y`: Negative of log transform of the bioactivity value.\n- `split`: Train-test split based on activity cliff.\n\n## Data resource:\n- [Exposing the Limitations of Molecular Machine Learning with Activity Cliffs\n](https://pubs.acs.org/doi/10.1021/acs.jcim.2c01073)\n- Github: https://github.com/molML/MoleculeACE'
    
    path = f"org-MolecularML/moleculeACE/docs/{dataset_name}_readme.md"
    with dm.fs.fsspec.open(path, "w") as f:
        f.write(readme)
    

In [16]:
for path in dm.fs.glob(
    f"gs://polaris-public/polaris-recipes/org-MolecularML/moleculeace/data/raw/*.csv"
):
    table = dm.read_csv(path)
    name = pathlib.Path(path).stem
    chembl_id, readout = name.split("_")
    target_name = mapping[chembl_id]
    dataset_name = f"moleculeace_{name}"
    benchmark_name = dataset_name
    chembl_id, readout = name.split("_")

    print(dataset_name)

    annotations = {
        "smiles": ColumnAnnotation(
            description="Molecule SMILES string.",
            modality="molecule",
        ),
        "exp_mean [nM]": ColumnAnnotation(
            description="Bioactivity in concentration nano Molar",
        ),
        "y": ColumnAnnotation(
            description=f"Negative of log transform of the bioactivity value {readout}",
            userAttributes={"unit": "nM"},
        ),
        "cliff_mol": ColumnAnnotation(
            description="Whether the molecule exibit activity cliff in the dataset.",
        ),
        "split": ColumnAnnotation(
            description="Train-test split provided by MoleculeACE.",
        ),
    }
    dataset = Dataset(
        table=table[annotations.keys()].copy(),
        name=dataset_name,
        description=f"Bioassay {chembl_id} of protein {target_name} molecular machine learning with activity cliffs.",
        source=source,
        annotations=annotations,
        owner=owner,
        tags=tags,
        license="CC-BY-4.0",
        readme=load_readme(f"org-MolecularML/moleculeACE/docs/{dataset_name}.md")
    )

    dataset.upload_to_hub()

    split = (
        table.query("split == 'train'").index.tolist(),
        table.query("split == 'test'").index.tolist(),
    )

    benchmark = SingleTaskBenchmarkSpecification(
        name=benchmark_name,
        dataset=dataset,
        target_cols=target_col,
        target_type=task,
        input_cols=[mol_col],
        split=split,
        metrics=["mean_squared_error"],
        tags=["ADMET", "singletask"],
        owner=owner,
        readme=load_readme(f"org-MolecularML/moleculeACE/docs/{dataset_name}.md")
    )

    benchmark.upload_to_hub()

moleculeace_CHEMBL1862_Ki
⠋ Uploading dataset...

2024-07-24 12:03:06.716 | INFO     | polaris._mixins:md5sum:27 - Computing the checksum. This can be slow for large datasets.


✅ SUCCESS: Your dataset has been successfully uploaded to the Hub. View it here: https://polarishub.io/datasets/molecularml/moleculeace_CHEMBL1862_Ki
 
⠋ Uploading benchmark...

/mnt/ps/home/CORP/lu.zhu/miniconda3/envs/po_datasets/lib/python3.12/site-packages/yaspin/core.py:228: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value
2024-07-24 12:03:07.926 | INFO     | polaris._mixins:md5sum:27 - Computing the checksum. This can be slow for large datasets.


✅ SUCCESS: Your benchmark has been successfully uploaded to the Hub. View it here: https://polarishub.io/benchmarks/molecularml/moleculeace_CHEMBL1862_Ki
 
moleculeace_CHEMBL1871_Ki
⠋ Uploading dataset...

...

/mnt/ps/home/CORP/lu.zhu/miniconda3/envs/po_datasets/lib/python3.12/site-packages/yaspin/core.py:228: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value
2024-07-24 12:03:51.685 | INFO     | polaris._mixins:md5sum:27 - Computing the checksum. This can be slow for large datasets.


✅ SUCCESS: Your dataset has been successfully uploaded to the Hub. View it here: https://polarishub.io/datasets/molecularml/moleculeace_CHEMBL4792_Ki
 
⠋ Uploading benchmark...

/mnt/ps/home/CORP/lu.zhu/miniconda3/envs/po_datasets/lib/python3.12/site-packages/yaspin/core.py:228: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value
2024-07-24 12:03:52.736 | INFO     | polaris._mixins:md5sum:27 - Computing the checksum. This can be slow for large datasets.


✅ SUCCESS: Your benchmark has been successfully uploaded to the Hub. View it here: https://polarishub.io/benchmarks/molecularml/moleculeace_CHEMBL4792_Ki
 


/mnt/ps/home/CORP/lu.zhu/miniconda3/envs/po_datasets/lib/python3.12/site-packages/yaspin/core.py:228: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value
